In [ ]:
import torch  
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import os 
import cv2
from pathlib import Path
import torchvision

In [14]:
image_dir = Path("saved_frames")

In [15]:
image_paths = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))

In [16]:
def load_and_preprocess_image(img_paths, target_size=(244,244)):
    img = cv2.imread(str(img_paths))

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = cv2.resize(img, target_size)
    
    img = img / 255.0
    
    return img 



In [17]:
images = [load_and_preprocess_image(img_path) for img_path in image_paths]
images = np.array(images)

In [20]:
dataset = tf.data.Dataset.from_tensor_slices(images)
dataset = dataset.batch(32)  # Batch size of 32 for processing